In [26]:
import pandas as pd

In [27]:
# read clean hawaii surf measurements from csv to dataframe
csvfile = "./clean_hawaii_measurements.csv"
measurements_df = pd.read_csv(csvfile,dtype=object)
measurements_df.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0,63
2,USC00519397,2010-01-03,0,74
3,USC00519397,2010-01-04,0,76
4,USC00519397,2010-01-07,0.06,70


In [28]:
# read hawaii station information from csv to dataframe
csvfile = "./hawaii_stations.csv"
stations_df = pd.read_csv(csvfile,dtype=object)
stations_df.head()

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.2716,-157.8168,3
1,USC00513117,"KANEOHE 838.1, HI US",21.4234,-157.8015,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.5213,-157.8374,7
3,USC00517948,"PEARL CITY, HI US",21.3934,-157.9751,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.4992,-158.0111,306.6


In [29]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float

In [30]:
# create an engine to sqlite db
engine = create_engine("sqlite:///hawaii.sqlite")

In [31]:
# Create a connection to the engine called conn
conn = engine.connect()

In [32]:
# Use `declarative_base` from SQLAlchemy to connect your class to your sqlite database
Base = declarative_base()

In [33]:
class Measurement (Base):
    __tablename__ = 'measurements'
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    date = Column(Text)
    prcp = Column(Float)
    tobs = Column(Integer)
    
    def __repr__(self):
        return f"station={self.station}, date={self.date}, prcp={self.prcp}, tobs={self.tobs}"
    

In [34]:
class Station (Base):
    __tablename__ = 'stations'
    id = Column(Integer, primary_key=True)
    station = Column(Text)
    name = Column(Text)
    longitude = Column(Float)
    latitude = Column(Float)
    elevation = Column(Float)
    
    def __repr__(self):
        return f"station={self.station}, name={self.name}, longitude={self.longitude}, latitude={self.latitude}, elevation={self.elevation}"
    

In [35]:
# Use `create_all` to create the positions table in the database
Base.metadata.create_all(engine)

In [36]:
# Use MetaData from SQLAlchemy to reflect the tables\n",
metadata = MetaData(bind=engine)
metadata.reflect()

In [37]:
# Save the reference to the `measurements` table as a variable called `measurements`
# Save the reference to the `stations` table as a variable called `stations`
measurements_table = sqlalchemy.Table('measurements', metadata, autoload=True)
stations_table = sqlalchemy.Table('stations', metadata, autoload=True)

In [38]:
# Use `measurements.delete()` to remove any existing data.
conn.execute(measurements.delete())

In [39]:
# Use `stations.delete()` to remove any existing data.
conn.execute(stations.delete())

In [40]:
measurements_data = measurements_df.to_dict(orient='records')
conn.execute(measurements_table.insert(), measurements_data)

In [41]:
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65),
 (2, 'USC00519397', '2010-01-02', 0.0, 63),
 (3, 'USC00519397', '2010-01-03', 0.0, 74),
 (4, 'USC00519397', '2010-01-04', 0.0, 76),
 (5, 'USC00519397', '2010-01-07', 0.06, 70)]

In [42]:
stations_data = stations_df.to_dict(orient='records')
conn.execute(stations_table.insert(), stations_data)

In [43]:
conn.execute("select * from stations limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', -157.8168, 21.2716, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', -157.8015, 21.4234, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', -157.8374, 21.5213, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', -157.9751, 21.3934, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', -158.0111, 21.4992, 306.6)]

In [44]:
conn.close()